In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train_undersampling.csv')


In [3]:
dft = pd.read_csv('test.csv')

In [39]:
df.columns

Index(['Unnamed: 0', 'id', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length',
       'loan_status'],
      dtype='object')

In [4]:
a =df['person_home_ownership'].unique()

In [5]:
b = dft['person_home_ownership'].unique()

In [6]:
print(a, b)

['RENT' 'MORTGAGE' 'OWN' 'OTHER'] ['RENT' 'MORTGAGE' 'OWN' 'OTHER']


In [4]:
df['person_home_ownership'] = df['person_home_ownership'].map({'RENT' : 0, 'OWN': 1, 'MORTGAGE' : 2, 'OTHER' : 3})
df['loan_intent'] = df['loan_intent'].map({'EDUCATION' : 0, 'MEDICAL' : 1, 'PERSONAL' : 2, 'VENTURE' : 3, 'DEBTCONSOLIDATION' : 4, 'HOMEIMPROVEMENT' : 5})
df['loan_grade'] = df['loan_grade'].map({'B' : 0, 'C' : 1, 'A' : 2, 'D' : 3,'E' : 4, 'F' : 5, 'G' : 6})
dft['person_home_ownership'] = dft['person_home_ownership'].map({'RENT' : 0, 'OWN': 1, 'MORTGAGE' : 2, 'OTHER' : 3})
dft['loan_intent'] = dft['loan_intent'].map({'EDUCATION' : 0, 'MEDICAL' : 1, 'PERSONAL' : 2, 'VENTURE' : 3, 'DEBTCONSOLIDATION' : 4, 'HOMEIMPROVEMENT' : 5})
dft['loan_grade'] = dft['loan_grade'].map({'B' : 0, 'C' : 1, 'A' : 2, 'D' : 3,'E' : 4, 'F' : 5, 'G' : 6})

In [7]:
unknown_values = set(b) - set(a)
print("Unknown values in real-time data:", unknown_values)

Unknown values in real-time data: set()


In [40]:
from sklearn.preprocessing import LabelEncoder
label_encoder_home_ownership = LabelEncoder()
df['person_home_ownership'] = label_encoder_home_ownership.fit_transform(df['person_home_ownership'])
df['loan_intent'] = label_encoder_home_ownership.fit_transform(df['loan_intent'])  
df['loan_grade'] = label_encoder_home_ownership.fit_transform(df['loan_grade'])


In [41]:
dft['person_home_ownership'] = label_encoder_home_ownership.fit_transform(dft['person_home_ownership'])
dft['loan_intent'] = label_encoder_home_ownership.fit_transform(dft['loan_intent'])  
dft['loan_grade'] = label_encoder_home_ownership.fit_transform(dft['loan_grade'])

In [61]:
dft.shape

(0, 12)

In [102]:
dft.shape

(39098, 12)

In [103]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])  
df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])     

In [105]:
dft['person_home_ownership'] = label_encoder.transform(dft['person_home_ownership'])
dft['loan_intent'] = label_encoder.transform(dft['loan_intent'])
dft['loan_grade'] = label_encoder.transform(dft['loan_grade'])

In [29]:
uniq1 = df['person_home_ownership'].unique()
uniq2 = df['loan_intent'].unique()
uni3  = df['loan_grade'].unique()
print(uniq1, uniq2, uni3)

[3 0 2 1] [4 5 2 3 1 0] [1 2 0 3 4 6 5]


In [5]:
X_train = df[['person_age', 'person_income', 'person_home_ownership',
         'loan_intent', 'loan_grade', 'loan_amnt',
         'loan_int_rate', 'loan_percent_income']]
y = df['loan_status'].values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [8]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.3, random_state=42)

In [29]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

e:\AI_Projek\Compettion_Load_Predict\bakso2\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=74, batch_size=24, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

Epoch 1/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5738 - loss: 0.6653 - val_accuracy: 0.8293 - val_loss: 0.4637
Epoch 2/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8358 - loss: 0.4325 - val_accuracy: 0.8362 - val_loss: 0.3954
Epoch 3/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8440 - loss: 0.3812 - val_accuracy: 0.8375 - val_loss: 0.3790
Epoch 4/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8397 - loss: 0.3839 - val_accuracy: 0.8422 - val_loss: 0.3718
Epoch 5/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8398 - loss: 0.3778 - val_accuracy: 0.8413 - val_loss: 0.3668
Epoch 6/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8564 - loss: 0.3556 - val_accuracy: 0.8413 - val_loss: 0.3658
Epoch 7/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8494 - loss: 0.3590 - val_accuracy: 0.8409 - val_loss: 0.3662
Epoch 8/74
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8442 - loss: 0.3603 - val_accuracy: 0.

In [31]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)

# Menggunakan threshold 0.5 untuk binarize hasil prediksi
predictions_binary = (predictions > 0.5).astype(int)

# Menghitung classification report
print("Classification Report:")
print(classification_report(y_test, predictions_binary))

# Menghitung confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions_binary))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      2488
           1       0.89      0.82      0.85      2522

    accuracy                           0.86      5010
   macro avg       0.86      0.86      0.86      5010
weighted avg       0.86      0.86      0.86      5010

Confusion Matrix:
[[2220  268]
 [ 451 2071]]


In [32]:
X_rl = dft[['person_age', 'person_income', 'person_home_ownership',
                'loan_intent', 'loan_grade', 'loan_amnt',
                'loan_int_rate', 'loan_percent_income']]

In [33]:
X_scaled_rl = scaler.fit_transform(X_rl)
X_reshapedrl = X_scaled_rl.reshape((X_scaled_rl.shape[0], 1, X_scaled_rl.shape[1]))

In [34]:
# Melakukan prediksi
predictions = model.predict(X_reshapedrl)


1222/1222 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [ ]:
# Pastikan dataset untuk prediksi memiliki 39098 baris



In [63]:
from sklearn.metrics import classification_report, confusion_matrix

# Menampilkan classification report
print("Classification Report:")
print(classification_report(y_rl, predictions))

# Menampilkan confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_rl, predictions))


Classification Report:


NameError: name 'y_rl' is not defined

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt

# Menghitung confusion matrix
cm = confusion_matrix(y_rl, predictions)

# Menggambar confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Benign", "DDoS"], yticklabels=["Benign", "DDoS"])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'y_rl' is not defined

In [24]:
binary_predictions = (predictions > 0.5).astype(int)

print(binary_predictions)

[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [1]]


In [35]:
predictions_1d = predictions.flatten() 

In [36]:
print(predictions_1d)

[0.99528104 0.14040138 0.9933115  ... 0.11252937 0.947792   0.99535376]


In [56]:
dft.shape

(39098, 12)

In [120]:
p = len(dft['id'])
print(p)

39098


In [121]:
l = len(predictions_1d)
print(l)

39098


In [91]:
subset_id = dft['id'][:len(predictions_1d)]  # Ambil subset id yang sesuai dengan panjang prediksi

# Buat DataFrame hasil
results = pd.DataFrame({
    'id': subset_id,
    'loan_status': predictions_1d
})

# Simpan hasil ke CSV
results.to_csv('predictions.csv', index=False)

In [83]:
import seaborn as sns
import matplotlib.pyplot as plt
count_values = df3['loan_status'].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(x=count_values.index, y=count_values.values, palette='viridis')
plt.title('loan_status')
plt.xlabel('Value')
plt.ylabel('jumlah')
for i, v in enumerate(count_values.values):
    plt.text(i, v + 0.1, str(v), ha='center')

plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [37]:
results = pd.DataFrame({
    'id': dft['id'],
    'loan_status': predictions_1d
})

# Menyimpan hasil ke CSV
results.to_csv('Zoltrak8_Finish.csv', index=False)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

model2 = Sequential()

# Layer RNN pertama, mengembalikan seluruh urutan (3D)
model2.add(SimpleRNN(64, input_shape=(timesteps, features), return_sequences=True, activation='relu'))
model2.add(Dropout(0.2))

# Layer RNN kedua, juga harus menerima input 3D
model2.add(SimpleRNN(32, return_sequences=False, activation='relu'))  # Output 2D setelah ini

# Layer Dense untuk klasifikasi
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# Kompilasi dan pelatihan
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluasi model
loss, accuracy = model2.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


NameError: name 'timesteps' is not defined